In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
import seaborn as sns
import missingno as msno
import folium
from folium import plugins

In [ ]:
!pip3 install plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

In [ ]:
plt.rcParams['figure.figsize'] = (20,10)
plt.rcParams.update({'font.size': 12})
mpl.rcParams['axes.unicode_minus'] = False

In [ ]:
df = pd.read_csv('../input/suicide-rates-overview-1985-to-2016/master.csv')

suicide_no = 자살한 사람 수 

suicides/100k pop = 10만 명 당 자살한 사람 수

HDI for year = 인간 개발 지수



In [ ]:
df

In [ ]:
df.info()

In [ ]:
msno.bar(df,figsize=(12,7))

In [ ]:
df.hist(edgecolor='black')

plt.show()

In [ ]:
sns.heatmap(df.corr(), annot = True, linewidths = 2, cmap = 'Greens')

각 컬럼 간의 상관관계를 보면 suicides_no는 population과 강한 양의 상관관계를 보이고 있습니다.

또한 year과는 약한 음의 상관관계를 보여줍니다.


In [ ]:
fig = px.scatter(df, 
                 x = 'population',
                 y = 'suicides_no')

fig.show()

population이 낮은 경우 suicides_no가 낮은편에 속하지만 

population이 높은 경우 suicides_no가 증가하는 것을 확인 가능합니다.

In [ ]:
fig = px.scatter(df, 
                 x = 'year',
                 y = 'suicides_no')

fig.show()

그래프를 보면 suicides_no 최고치는 1994년에 기록된 것으로 확인됩니다.
***

In [ ]:
fig = px.box(df,y = 'suicides_no')
fig.show()

In [ ]:
fig = px.box(df, y = 'suicides/100k pop')

fig.show()

***

In [ ]:
# 단계구분도를 통한 연도별 suicides_no
df_iso_alpha = px.data.gapminder()
df_iso_alpha.drop(['pop','gdpPercap'],axis=1,inplace=True)
df_iso = pd.merge(df, df_iso_alpha, on=['country'],how='inner')
df_iso = df_iso[['country','year_x','population','suicides_no','iso_alpha']]

In [ ]:
df_iso.head()

In [ ]:
fig = px.choropleth(df_iso,
                    locations= 'iso_alpha', 
                    color= 'suicides_no', 
                    hover_name= 'country',
                    animation_frame= 'year_x', 
                    range_color=[0,100],
                    )
fig.show()

***

In [ ]:
# 연도별 suicides_no
by_year_no = pd.DataFrame(df.groupby('year')['suicides_no'].sum())
by_year_no.head()

In [ ]:
fig = go.Figure(go.Scatter(x = by_year_no.index, 
                           y = by_year_no['suicides_no'],
                           mode = 'lines+markers'))

fig.show()

1988년부터 1990년까지 suicides_no가 급격하게 상승합니다.

1999년에 가장 높은 suicides_nos에 이릅니다.

2003년 이후 조금씩 suicides_no이 낮아집니다.

2014년 이후에는 suicides_no이 급격히 낮아집니다.
***

In [ ]:
# 연도별 suicides/100k pop
by_year_100k = pd.DataFrame(df.groupby('year')['suicides/100k pop'].sum())
by_year_100k.head()

In [ ]:
fig = go.Figure(go.Scatter(x = by_year_100k.index,
                           y = by_year_100k['suicides/100k pop'],
                           mode = 'lines+markers'))

fig.show()

suicide_no와는 다르게 1995년에 가장 높은 suicides/100k pop에 이릅니다.

1995년 이후 지속적으로 suicides/100k pop이 낮아집니다.

2014년 이후에는 suicides/100k pop이 급격히 낮아지면서 2016년에 최저 suicides/100k pop에 이릅니다.

***
자료의 통일성을 위해서 suicide_no로 통일하겠습니다.
***

In [ ]:
# 성별에 따른 suicide_no
by_sex = pd.DataFrame(df.groupby(['sex'])['suicides_no'].sum())
by_sex['suicides_no per'] = (by_sex['suicides_no'] / sum(by_sex['suicides_no']))*100
by_sex

In [ ]:
fig = make_subplots(rows = 1, cols = 2 , specs=[[{'type':'xy'}, {'type':'domain'}]])

fig.add_trace(go.Bar(x = by_sex.index,
                     y = by_sex['suicides_no'],
                     name = 'Suicides No',
                     marker_color = px.colors.qualitative.Plotly),1,1)

fig.add_trace(go.Pie(labels = by_sex.index,
                     values = by_sex['suicides_no per'],
                     name = 'Suicides No Per',
                     marker_colors = px.colors.qualitative.Plotly,
                     pull = [0.2,0]),1,2)

fig.update_layout(title_text='Number/Percent rate By Sex',
                  title_font_size=20)

fig.show()

남성의 suicides_no가 76.9%로 여성의 suicides_no보다 약 3배 이상 높습니다.

***

In [ ]:
# 세대 별 suicide_no
by_gener = pd.DataFrame(df.groupby('generation')['suicides_no'].sum())
by_gener['suicides_no per'] = (by_gener['suicides_no'] / sum(by_gener['suicides_no'])) * 100
by_gener

In [ ]:
fig = make_subplots(rows = 1, cols = 2, specs = [[{'type':'xy'},{'type':'domain'}]])

fig.add_trace(go.Bar(x = by_gener.index,
                     y = by_gener['suicides_no'],
                     name = 'Suicides No',
                     marker_color = px.colors.qualitative.Pastel),1,1)

fig.add_trace(go.Pie(labels = by_gener.index,
                     values = by_gener['suicides_no per'],
                     name = 'Suicides No per',
                     marker_colors = px.colors.qualitative.Pastel,
                     pull = [0.2,0,0,0.4,0,0]),1,2)       
             
fig.update_layout(title_text='Number/Percent rate By Generation',
                  title_font_size=20)

fig.show()

Boommer 세대의 경우 33.9%로 가장 높습니다

Z 세대의 경우 0.236%로 가장 적은 수치를 보여줍니다.

X, Slient 세대의 경우 각각 22.7%, 26.4%로 상당 부분을 차지하고 있습니다

G.I, Milenials 세대의 경우 각각 7.56%, 9.24%로 비교적 낮은 수치를 보여줍니다.

In [ ]:
by_gener_sex = pd.DataFrame(df.groupby(['generation','sex'])['suicides_no'].sum())
by_gener_sex['suicides_no per'] = (by_gener_sex['suicides_no'] / sum(by_gener_sex['suicides_no'])) * 100
by_gener_sex.reset_index('sex',inplace=True)
by_gener_sex

In [ ]:
fig = px.bar(by_gener_sex,
             x = by_gener_sex.index,
             y = 'suicides_no per',
             color = 'sex')

fig.show()

모든 generation에서 남자의 사망 비율이 훨씬 높은 것을 확인 가능합니다

Slient 세대에서는 여성의 suicidies_no 비율이 6.99%로 가장 높은 비율을 차지했습니다.
***

In [ ]:
# 나이 별 suicide_no
by_age = pd.DataFrame(df.groupby('age')['suicides_no'].sum())
by_age['suicides_no per'] = (by_age['suicides_no'] / sum(by_age['suicides_no'])) * 100
by_age

In [ ]:
fig = make_subplots(rows = 1, cols = 2, specs = [[{'type':'xy'},{'type':'domain'}]])

fig.add_trace(go.Bar(x = by_age.index,
                     y = by_age['suicides_no'],
                     name = 'Suicides No',
                     marker_color = px.colors.qualitative.Vivid),1,1)

fig.add_trace(go.Pie(labels = by_age.index,
                     values = by_age['suicides_no per'],
                     name = 'Suicides No per',
                     pull = [0,0,0,0.4,0,0.2],
                     marker_colors = px.colors.qualitative.Vivid),1,2)

fig.update_layout(title_text = 'Count/Percent rate by Age',
                  title_font_size = 20)

fig.show()

35-54 year의 경우 36.3%로 가장 많습니다.

5-14 years의 경우 0.774%로 가장 적습니다.

In [ ]:
by_age_sex = pd.DataFrame(df.groupby(['age','sex'])['suicides_no'].sum())
by_age_sex['suicides_no per'] = (by_age_sex['suicides_no'] / sum(by_age_sex['suicides_no'])) * 100
by_age_sex.reset_index('sex',inplace=True)
by_age_sex

In [ ]:
fig = px.bar(by_age_sex,
             x = by_age_sex.index,
             y = 'suicides_no per',
             color = 'sex')

fig.show()

모든 age에서 남자의 사망 비율이 훨씬 높은 것을 확인 가능합니다

***

In [ ]:
# suicide_no top10 나라
df_country = pd.DataFrame(df.groupby('country')['suicides_no'].sum())
df_country.sort_values('suicides_no',ascending=False, inplace=True)
df_country_top10 = df_country[:10]
df_country_top10

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(x = df_country_top10.index,
                     y = df_country_top10['suicides_no'],
                     marker_color = px.colors.qualitative.D3))

fig.update_layout(title_text='Suicide Top10 Country',
                  title_font_size=20)

fig.show()

1위는 120만명을 기록한 러시아, 2위는 100만명을 기록한 미국, 3위는 80만명을 기록한 일본이 차지했습니다.

3위인 일본의 suicides_no 가 80만명이지만, 4위인 프랑스는 33만명으로 일본 suicides_no에 절반도 미치지 못하는 수로 격차가 많이 나는 것을 볼 수 있습니다.

한국의 경우 26만명으로 7위에 해당하는 모습을 확인할 수 있습니다.

In [ ]:
# 나라 별 suicide_no 추이
country = pd.DataFrame(df.groupby(['country','year'])['suicides_no'].sum())
country.reset_index(inplace=True)
country

In [ ]:
plt.figure(figsize=(20,30))
plt.rcParams['axes.facecolor'] = 'black'

plt.subplot(3,1,1)
sns.barplot(data = country[country['country']=='Russian Federation'],
            x = 'year', 
            y = 'suicides_no',
            palette = 'Blues')
plt.title('Suicide Trand in Russian Federation',fontsize=20)

plt.subplot(3,1,2)
sns.barplot(data = country[country['country']=='United States'],
            x = 'year', 
            y = 'suicides_no',
            palette = 'Greens')
plt.title('Suicide Trand in United States',fontsize=20)

plt.subplot(3,1,3)
sns.barplot(data = country[country['country']=='Japan'],
            x = 'year', 
            y = 'suicides_no',
            palette = 'Purples')
plt.title('Suicide Trand in Japan',fontsize=20)


plt.show()

1위인 러시아의 경우 1994년에 suicides_no 최고치를 기록했습니다.

2000년 이후에는 지속적으로 감소하는 그래프를 보여줍니다.

2위인 미국의 경우 1985년부터 1998년까지 비슷한 수치를 보여줍니다.

하지만 1999년에 suicides_no가 약간 감소한 이후 지속적으로 상승하는 그래프를 보여줍니다.

3위인 일본의 경우 1990년, 1991년에 suicides_no가 가장 낮은 수치를 보여줍니다.

하지만 1998년에는 자살 인원의 급격한 상승으로 인해 자살 최고치를 기록합니다.

2009년 이후에는 감소하는 그래프를 보여줍니다.
***

In [ ]:
# 러시아 나이/성별/세대 별 suicides_no

russia = pd.DataFrame(df[df['country']=='Russian Federation'])
russia.head()

In [ ]:
def russia_suicide(x):
    x = pd.DataFrame(russia.groupby(x)['suicides_no'].sum())
    x.reset_index(inplace=True)
    x['percent'] = (x['suicides_no']/sum(x['suicides_no']))*100
    return x

In [ ]:
russia_suicide('age')

In [ ]:
# Sex
fig =  make_subplots(rows = 1, cols = 2, specs =[[{'type':'xy'},{'type':'domain'}]])

fig.add_trace(go.Bar(x = russia_suicide('sex')['sex'],
                     y = russia_suicide('sex')['suicides_no'],
                     name = 'Sex',
                     marker_color = px.colors.qualitative.Plotly),1,1)

fig.add_trace(go.Pie(labels = russia_suicide('sex')['sex'],
                     values = russia_suicide('sex')['percent'],
                     pull = [0,0.2],
                     name = 'Sex',
                     marker_colors = px.colors.qualitative.Plotly),1,2)
fig.show()

# Generation
fig =  make_subplots(rows = 1, cols = 2, specs =[[{'type':'xy'},{'type':'domain'}]])

fig.add_trace(go.Bar(x = russia_suicide('generation')['generation'],
                     y = russia_suicide('generation')['suicides_no'],
                     name = 'Generation',
                     marker_color = px.colors.qualitative.Pastel),1,1)

fig.add_trace(go.Pie(labels = russia_suicide('generation')['generation'],
                     values = russia_suicide('generation')['percent'],
                     pull = [0.2,0,0,0.2,0,0],
                     name = 'Generation',
                     marker_colors = px.colors.qualitative.Pastel),1,2)
fig.show()

# Age
fig =  make_subplots(rows = 1, cols = 2, specs =[[{'type':'xy'},{'type':'domain'}]])

fig.add_trace(go.Bar(x = russia_suicide('age')['age'],
                     y = russia_suicide('age')['suicides_no'],
                     name = 'Age',
                     marker_color = px.colors.qualitative.Vivid),1,1)

fig.add_trace(go.Pie(labels = russia_suicide('age')['age'],
                     values = russia_suicide('age')['percent'],
                     pull = [0,0,0.2,0.2,0,0],
                     name = 'Age',
                     marker_colors = px.colors.qualitative.Vivid),1,2)
fig.show()

전체와 러시아를 비교하면 sex의 경우 남성의 suicides_no의 비율이 5%가량 증가했습니다

age의 경우 25-34 years와 35-54 years는 3% 가량 상승했고 

generation의 경우 Boomers 세대는 33.9.%에서 38.9%로 5% 상승을 했습니다.

Slient, G.I 세대의 경우에는 2~3% 가량 감소했습니다

X, Millenial, Z 세대의 경우에는 1%미만으로 변동이 거의 없습니다.

55-74 years와 75+ years는 2% ~ 3% 가량 하락했습니다

그외 나이는 거의 변동이 없습니다

***
머신러닝을 이용한 정확도 예측

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df_ml = df[['year','sex','age','suicides_no','population','generation']]
df_ml_label = df_ml
df_ml_label

In [ ]:
le = LabelEncoder()

df_ml_label['age'] = le.fit_transform(df_ml['age'])
df_ml_label['sex'] = le.fit_transform(df_ml['sex'])
df_ml_label['generation'] = le.fit_transform(df_ml['generation'])

df_ml_label

In [ ]:
X = df_ml_label.drop('suicides_no',axis=1)
y = df_ml_label['suicides_no']

print(X.shape)
print(y.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=100)

In [ ]:
# Linear Regression
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train,y_train)
pred = model.predict(X_test)

linear_mse = mean_squared_error(y_test, pred)
linear_rmse = np.sqrt(linear_mse)
linear_r2 = r2_score(y_test,pred)

In [ ]:
print('MSE : ', linear_mse)
print('RMSE : ', linear_rmse)
print('R2 score : ', linear_r2)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()
model.fit(X_train,y_train)
pred = model.predict(X_test)

rf_mse = mean_squared_error(y_test, pred)
rf_rmse = np.sqrt(rf_mse)
rf_r2 = r2_score(y_test,pred)

In [ ]:
print('MSE : ', rf_mse)  
print('RMSE : ', rf_rmse)
print('R2 score : ', rf_r2)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor()
model.fit(X_train,y_train)
pred = model.predict(X_test)

dt_mse = mean_squared_error(y_test, pred)
dt_rmse = np.sqrt(dt_mse)
dt_r2 = r2_score(y_test,pred)

In [ ]:
print('MSE : ',dt_mse)
print('RMSE : ', dt_rmse)
print('R2 score : ', dt_r2)

In [ ]:
ml = pd.DataFrame([['Linear Regression',linear_rmse,linear_r2],
                  ['Random Forest',rf_rmse,rf_r2],
                  ['Decision Tree',dt_rmse,dt_r2]])
ml.columns = ['Model','RMSE','R2 Score']
ml

In [ ]:
fig = px.bar(ml,
             x = 'Model',
             y = 'RMSE',
             color = 'Model')

fig.update_layout(title_text = 'RMSE',
                  title_font_size = 20)

fig.show()

fig = px.bar(ml,
             x = 'Model',
             y = 'R2 Score',
             color = 'Model')

fig.update_layout(title_text = 'R2 Score',
                  title_font_size = 20)

RMSE가 가장 낮고 R2 Score가 가장 높은 Random Forest가 예측도가 가장 높습니다

***
Min Max Scaler를 이용한 머신러닝

In [ ]:
df_ml = df[['year','sex','age','suicides_no','population','generation']]
df_ml_label = df_ml
df_ml_label

In [ ]:
le = LabelEncoder()

df_ml_label['sex'] = le.fit_transform(df_ml['sex'])
df_ml_label['age'] = le.fit_transform(df_ml['age'])
df_ml_label['generation'] = le.fit_transform(df_ml['generation'])

df_ml_label

In [ ]:
X = df_ml_label.drop('suicides_no',axis=1)
y = df_ml_label['suicides_no']

print(X.shape)
print(y.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=100)

In [ ]:
minmax = MinMaxScaler()

X_train = minmax.fit_transform(X_train)
X_test = minmax.transform(X_test)

In [ ]:
# Linear Scaler (Min Max Scaler)

model = LinearRegression()
model.fit(X_train,y_train)
pred = model.predict(X_test)

ml_linear_mse = mean_squared_error(y_test,pred)
ml_linear_rmse = np.sqrt(ml_linear_mse)
ml_linear_r2 = r2_score(y_test,pred)

In [ ]:
print('MSE : ', ml_linear_mse)
print('RMSE : ', ml_linear_rmse)
print('R2 Score : ', ml_linear_r2)

In [ ]:
# Random Forest (Min Max Scaler)

model = RandomForestRegressor()
model.fit(X_train,y_train)
pred = model.predict(X_test)

ml_rf_mse = mean_squared_error(y_test,pred)
ml_rf_rmse = np.sqrt(ml_rf_mse)
ml_rf_r2 = r2_score(y_test,pred)

In [ ]:
print('MSE : ', ml_rf_mse)
print('RMSE : ', ml_rf_rmse)
print('R2 Score : ', ml_rf_r2)

In [ ]:
# Decision Tree(Min Max Scaler)

model = DecisionTreeRegressor()
model.fit(X_train,y_train)
pred = model.predict(X_test)

ml_dt_mse = mean_squared_error(y_test,pred)
ml_dt_rmse = np.sqrt(ml_dt_mse)
ml_dt_r2 = r2_score(y_test,pred)

In [ ]:
print('MSE : ', ml_dt_mse)
print('RMSE : ', ml_dt_rmse)
print('R2 Score : ', ml_dt_r2)

In [ ]:
mm_ml = pd.DataFrame([['Linear Regression',ml_linear_rmse,ml_linear_r2],
                     ['Random Forest',ml_rf_rmse,ml_rf_r2],
                     ['Decision Tree',ml_dt_rmse,ml_dt_r2]])
mm_ml.columns = ['Model','RMSE','R2 Score']
mm_ml

In [ ]:
fig = px.bar(mm_ml,
             x = 'Model',
             y = 'RMSE',
             color = 'Model')
fig.update_layout(title_text = 'Min Max Scaler RMSE',
                  title_font_size = 20)
fig.show()

fig = px.bar(mm_ml,
             x = 'Model',
             y = 'R2 Score',
             color = 'Model')
fig.update_layout(title_text = 'Min Max Scaler R2 Score',
                  title_font_size = 20)
fig.show()


Min Max Scaler를 통해 scaling을 적용한 결과 

RMSE가 가장 낮고 R2 Score가 가장 높은 Random Forest가 예측도가 가장 높습니다

*** 


In [ ]:
fig = go.Figure(data = [go.Bar(x = ml[ml['Model']=='Random Forest']['Model'],
                               y = ml[ml['Model']=='Random Forest']['RMSE'],
                               name = 'RMSE'),
                        go.Bar(x = mm_ml[mm_ml['Model']=='Random Forest']['Model'],
                               y = mm_ml[mm_ml['Model']=='Random Forest']['RMSE'],
                               name = 'Min Max Scaler RMSE')])

fig.show()

fig = go.Figure(data = [go.Bar(x = ml[ml['Model']=='Random Forest']['Model'],
                               y = ml[ml['Model']=='Random Forest']['R2 Score'],
                               name = 'R2 Score'),
                        go.Bar(x = mm_ml[mm_ml['Model']=='Random Forest']['Model'],
                               y = mm_ml[mm_ml['Model']=='Random Forest']['R2 Score'],
                               name = 'Min Max Scaler R2 Score')])

fig.show()

비교를 하면 Min Max Scaler를 통해 Scaling을 하지않은 모델의 RMSE와 R2 Score가 근소한 차이로 더 효율적입니다.

따라서 Scaling을 하지않은 Random Forest가 가장 예측도가 높다고 판단할 수 있습니다.